# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from pickle import load
import pickle
import DiadFit as pf
pf.__version__

'0.0.59'

In [2]:
MasterFolder = r"P:\WORK-GENERAL\POSTDOC-UCB\BERKELEY-VIBE\Documents\Projects\Data\Hawaii_FI\Data\Raman\Leilani-2018"
DayFolder = os.path.join(MasterFolder, "Leilani_2018FI_Oct2022")

meta_path=DayFolder +  '\Metadata'
spectra_path=DayFolder + '\Spectra'
filetype='headless_txt'

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')


output_path=MasterFolder +'\OUTPUT'


## Load in specra, and combine

In [3]:
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,LL4-11-FI#1-miserablediad_CRR_DiadFit,102.787874,0.216899,0.216899,1286.214767,0.216899,37.325696,1286.214767,78.817444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,LL4-14dp-FI#1rep1_CRR_DiadFit,103.419654,0.009445,0.006813,1284.979629,0.005675,2266.641420,1284.979629,4124.911310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,LL4-14dp-FI#1rep2_CRR_DiadFit,103.416352,0.008948,0.006447,1284.991877,0.005348,2301.210436,1284.991877,4231.892173,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,LL4-14dp-FI#1rep3,103.414283,0.008384,0.005992,1284.997517,0.004806,2371.517663,1284.997517,4316.406238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,LL4-14dp-FI#2_CRR_DiadFit,103.431244,0.009843,0.007102,1285.017453,0.005920,1339.170110,1285.017453,2420.406311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'Si', 'series','IMG','Cap','window','nodiad'],
 sort=True)
diad_meta

['LL4-11-FI#1-miserablediad.txt',
 'LL4-14dp-FI#1rep1.txt',
 'LL4-14dp-FI#1rep2.txt',
 'LL4-14dp-FI#1rep3.txt',
 'LL4-14dp-FI#2.txt',
 'LL4-14dp-MI#1rep1.txt',
 'LL4-14dp-MI#1rep2.txt',
 'LL4-14dp-MI#1rep3.txt',
 'LL4-14dp-MI#1rep4.txt',
 'LL4-14dp-MI#1rep5.txt',
 'LL4-14dp-MI#1rep6.txt',
 'LL4-14dp-MI#3.txt',
 'LL4-FI12b-FI#1.txt',
 'LL4-FI12b-FI#2.txt',
 'LL4-FI12b-FI#3cosmicrat.txt',
 'LL4-FI12b-MI#1.txt',
 'LL4-FI12b-MI#2-12mW.txt',
 'LL4-FI12b-MI#2-6mW.txt',
 'LL4-FI12b-MI#3.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|██████████| 19/19 [00:03<00:00,  6.18it/s]

Done


'LL4-14dp-FI#1rep1.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'LL4-14dp-FI#1rep1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
file_s

array(['LL4-11-FI#1-miserablediad', 'LL4-14dp-FI#1rep1',
       'LL4-14dp-FI#1rep2', 'LL4-14dp-FI#1rep3', 'LL4-14dp-FI#2',
       'LL4-14dp-MI#1rep1', 'LL4-14dp-MI#1rep2', 'LL4-14dp-MI#1rep3',
       'LL4-14dp-MI#1rep4', 'LL4-14dp-MI#1rep5', 'LL4-14dp-MI#1rep6',
       'LL4-14dp-MI#3', 'LL4-FI12b-FI#1', 'LL4-FI12b-FI#2',
       'LL4-FI12b-FI#3cosmicrat', 'LL4-FI12b-MI#1', 'LL4-FI12b-MI#2-12mW',
       'LL4-FI12b-MI#2-6mW'], dtype='<U25')

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
Ne_corr=pf.calculate_Ne_corr_std_err_values(pickle_str='polyfit_data.pkl', 
    new_x=df_combo['sec since midnight'], CI=0.67)
Ne_corr.head()


2


,time,preferred_values,lower_values,upper_values
0,57657.0,0.997695,0.997686,0.997703
1,46213.0,0.997775,0.997767,0.997784
2,46634.0,0.997770,0.997761,0.997778
3,46986.0,0.997765,0.997757,0.997773
4,51301.0,0.997725,0.997716,0.997733


In [12]:

split_err=pf.propagate_errors_for_splitting(Ne_corr, df_combo)
df_combo_out=df_combo.copy()
df_combo_out.insert(1, 'Corrected_Splitting', df_combo['Splitting']*Ne_corr['preferred_values'])
df_combo_out.insert(2, 'Corr_Split+1σ', df_combo_out['Corrected_Splitting']+split_err)
df_combo_out.insert(3, 'Corr_Split-1σ',df_combo_out['Corrected_Splitting']-split_err)
df_combo_out.insert(4, 'Corr_Split_1σ_val', split_err)

nm=os.path.basename(DayFolder)
df_combo_out.to_excel(output_path+'/'+nm+'_FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [13]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name_x,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb,Model_name_y
0,LL4-14dp-FI#1rep1.txt,1150.415611,1041.408086,623.423691,Spline,NaN,NaN,NaN,NaN
1,LL4-14dp-FI#1rep2.txt,1150.415611,1047.480173,616.150118,Spline,NaN,NaN,NaN,NaN
2,LL4-14dp-FI#1rep3.txt,1150.415611,1033.147638,642.756534,Spline,NaN,NaN,NaN,NaN
3,LL4-14dp-FI#2.txt,1150.504103,347.891925,218.485700,Spline,NaN,NaN,NaN,NaN
4,LL4-14dp-MI#1rep1.txt,1150.297622,502.138058,311.404505,Spline,NaN,NaN,NaN,NaN


In [14]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

good job, no duplicate file names


In [15]:
file_sec_phase

array(['LL4-14dp-FI#1rep1', 'LL4-14dp-FI#1rep2', 'LL4-14dp-FI#1rep3',
       'LL4-14dp-FI#2', 'LL4-14dp-MI#1rep1', 'LL4-14dp-MI#1rep2',
       'LL4-14dp-MI#1rep3', 'LL4-14dp-MI#1rep4', 'LL4-14dp-MI#1rep5',
       'LL4-14dp-MI#1rep6', 'LL4-FI12b-FI#1', 'LL4-FI12b-FI#2',
       'LL4-FI12b-FI#3cosmicrat', 'LL4-FI12b-MI#2-6mW'], dtype=object)

In [16]:
df_combo['name_for_matching']

0     LL4-11-FI#1-miserablediad
1             LL4-14dp-FI#1rep1
2             LL4-14dp-FI#1rep2
3             LL4-14dp-FI#1rep3
4                 LL4-14dp-FI#2
5             LL4-14dp-MI#1rep1
6             LL4-14dp-MI#1rep2
7             LL4-14dp-MI#1rep3
8             LL4-14dp-MI#1rep4
9             LL4-14dp-MI#1rep5
10            LL4-14dp-MI#1rep6
11                LL4-14dp-MI#3
12               LL4-FI12b-FI#1
13               LL4-FI12b-FI#2
14      LL4-FI12b-FI#3cosmicrat
15               LL4-FI12b-MI#1
16          LL4-FI12b-MI#2-12mW
17           LL4-FI12b-MI#2-6mW
Name: name_for_matching, dtype: object

In [17]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo_out.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo_out

### Add some more useful columns

In [18]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
    
df_combo_sec_phase.head()

,Unnamed: 0,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,Corr_Split_1σ_val,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,...,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name_x,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb,Model_name_y,Carb_Diad_Ratio,SO2_Diad_Ratio
0,0,102.550907,102.767308,102.334506,0.216401,LL4-11-FI#1-miserablediad_CRR_DiadFit,102.787874,0.216899,0.216899,1286.214767,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,103.189573,103.196425,103.182720,0.006853,LL4-14dp-FI#1rep1_CRR_DiadFit,103.419654,0.009445,0.006813,1284.979629,...,1150.415611,1041.408086,623.423691,Spline,NaN,NaN,NaN,NaN,NaN,0.101077
2,0,103.185694,103.192184,103.179204,0.006490,LL4-14dp-FI#1rep2_CRR_DiadFit,103.416352,0.008948,0.006447,1284.991877,...,1150.415611,1047.480173,616.150118,Spline,NaN,NaN,NaN,NaN,NaN,0.099336
3,0,103.183165,103.189205,103.177125,0.006040,LL4-14dp-FI#1rep3,103.414283,0.008384,0.005992,1284.997517,...,1150.415611,1033.147638,642.756534,Spline,NaN,NaN,NaN,NaN,NaN,0.095665
4,0,103.195894,103.203031,103.188757,0.007137,LL4-14dp-FI#2_CRR_DiadFit,103.431244,0.009843,0.007102,1285.017453,...,1150.504103,347.891925,218.485700,Spline,NaN,NaN,NaN,NaN,NaN,0.057072


In [19]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_FI_fitting_w_sec_phases.xlsx')

In [20]:
from winotify import Notification, audio

toast= Notification(app_id="VSCode",title="Notebook completed",
                    msg="Step4_Stitch_Outputs_Together is done!",
                    duration="short")
toast.set_audio(audio.Mail,loop=False)
toast.show()